In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from scripts.load_data import *
from scripts.availabledata import *
from scripts.individual import *
from scripts.cumulative import *
from scripts.bothdatasets import *
from scripts.dataoption import *
from scripts.higher_years import *

In [3]:
configuration = load_configuration("../configuration/configuration.json")

data_individual, data_cumulative, data_student_numbers_first_years, data_student_numbers_higher_years, data_student_numbers_volume, data_latest, data_distances, ensemble_weights = load_data(configuration, StudentYearPrediction.FIRST_YEARS)

dataholder = Cumulative(data_cumulative, data_student_numbers_first_years, configuration)

data_cumulative = dataholder.preprocess()

numerus_fixus_list = configuration["numerus_fixus"]

c:\Users\jjble\Documents\Master\Jaar2_Semester2\Research_Internship\Tijdens\Code\studentprognose\scripts\load_data.py:11: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  data_cumulative = pd.read_csv(paths["path_cumulative"], sep=';', skiprows=[1]) if (paths["path_cumulative"] != "" and os.path.exists(paths["path_cumulative"])) else None
c:\Users\jjble\Documents\Master\Jaar2_Semester2\Research_Internship\Tijdens\Code\studentprognose\scripts\cumulative.py:72: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data[key] = data[key].str.replace('.', '')


In [4]:
predict_year = 2023
skip_years = 1
predict_week = 15

In [5]:
data_to_predict = data_cumulative[(data_cumulative["Collegejaar"] == predict_year - skip_years) &
                                               (data_cumulative["Weeknummer"] == predict_week)]

# dataholder.prepare_data()

# full_data = dataholder.data_cumulative.copy()

# full_data = full_data[full_data["Collegejaar"] >= 2016]
# full_data = full_data[full_data["Collegejaar"] <= predict_year]

# data = transform_data(full_data.copy(), 'ts')

data = pd.read_excel("full_data.xlsx")

In [6]:
total_mae = 0.0
total_mape = 0.0
count = 0.0

for _, row in data_to_predict.iterrows():
    programme = row['Croho groepeernaam']
    herkomst = row["Herkomst"]
    examtype = row["Examentype"]

    all_years = np.sort(data["Collegejaar"].unique())
    second_year = all_years[1]

    # Train/test split
    if programme not in numerus_fixus_list:
        train = data[(data["Collegejaar"] < predict_year - skip_years - 1) & (data["Examentype"] == examtype) & (
            ~data['Croho groepeernaam'].isin(numerus_fixus_list))]
        train_copy = data[(data["Collegejaar"] < predict_year - 1) & (data["Examentype"] == examtype) & (
            ~data['Croho groepeernaam'].isin(numerus_fixus_list))]
    elif programme in numerus_fixus_list:
        train = data[(data["Collegejaar"] < predict_year - skip_years - 1) & (data['Croho groepeernaam'] == programme)]
        train_copy = data[(data["Collegejaar"] < predict_year - 1) & (data['Croho groepeernaam'] == programme)]

    test = data[(data["Collegejaar"] == predict_year - skip_years) & (data['Croho groepeernaam'] == programme) & (data["Herkomst"] == herkomst)]

    if data_student_numbers_first_years is not None:
        train = train.merge(data_student_numbers_first_years[['Croho groepeernaam', 'Collegejaar', 'Herkomst', 'Aantal_studenten']],
                            on=['Croho groepeernaam', 'Collegejaar', 'Herkomst'])
        train_copy = train_copy.merge(data_student_numbers_first_years[['Croho groepeernaam', 'Collegejaar', 'Herkomst', 'Aantal_studenten']],
                            on=['Croho groepeernaam', 'Collegejaar', 'Herkomst'])

    train = train.drop_duplicates()
    train_copy = train_copy.drop_duplicates()

    train_copy["Collegejaar"] = train_copy["Collegejaar"] - skip_years
    train_copy = train_copy.rename(columns={"Aantal_studenten": "Aantal_studenten_next_year"})

    new_train = train.merge(train_copy[['Collegejaar', 'Examentype', 'Faculteit', 'Croho groepeernaam', 'Herkomst', 'Aantal_studenten_next_year']],
                            on=['Collegejaar', 'Examentype', 'Faculteit', 'Croho groepeernaam', 'Herkomst'], how='left')

    X_train = new_train.drop(['Aantal_studenten_next_year', "Aantal_studenten"], axis=1)
    y_train = new_train.pop('Aantal_studenten_next_year').fillna(0)

    # Encode
    # Specify the numeric and categorical column names
    numeric_cols = ['Collegejaar'] + [str(x) for x in get_weeks_list(38)]
    categorical_cols = ['Examentype', 'Faculteit', 'Croho groepeernaam', 'Herkomst']

    # Create transformers for numeric and categorical columns
    numeric_transformer = "passthrough"  # No transformation for numeric columns
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')

    # Create the column transformer
    preprocessor = ColumnTransformer(
        transformers=[
            ('numeric', numeric_transformer, numeric_cols),
            ('categorical', categorical_transformer, categorical_cols)
        ])

    # Apply the preprocessing to the training and test data
    X_train = preprocessor.fit_transform(X_train)
    test = preprocessor.transform(test)
    # Model
    model = XGBRegressor(learning_rate=0.25)

    model.fit(X_train, y_train)

    predictie = model.predict(test)

    final_prediction = int(round(predictie[0], 0))

    nr_of_students = data_student_numbers_first_years[(data_student_numbers_first_years["Collegejaar"] == predict_year) & (data_student_numbers_first_years["Croho groepeernaam"] == programme) & 
                                                      (data_student_numbers_first_years["Herkomst"] == herkomst)]["Aantal_studenten"]
    
    mae = "???"
    mape = "???"
    if len(nr_of_students) > 0:
        mae = abs(nr_of_students.iloc[0] - final_prediction)
        mape = abs((nr_of_students.iloc[0] - final_prediction) / nr_of_students.iloc[0])

        total_mae += mae
        total_mape += mape
        count += 1.0

    print(f"{programme}, {herkomst}, {final_prediction}, MAE: {mae}, MAPE: {mape}")

print(f"Final MAE: {total_mae / count}")
print(f"Final MAPE: {total_mape / count}")

B Algemene Cultuurwetenschappen, EER, 15, MAE: 1, MAPE: 0.07142857142857142
B Algemene Cultuurwetenschappen, NL, 25, MAE: 4, MAPE: 0.13793103448275862
B Algemene Cultuurwetenschappen, Niet-EER, 2, MAE: 2, MAPE: 0.5
B Artificial Intelligence, EER, 99, MAE: 52, MAPE: 1.1063829787234043
B Artificial Intelligence, NL, 99, MAE: 54, MAPE: 0.35294117647058826
B Artificial Intelligence, Niet-EER, 19, MAE: 4, MAPE: 0.26666666666666666
B Artificial Intelligence, onbekend, 9, MAE: ???, MAPE: ???
B Bedrijfskunde, EER, 19, MAE: 10, MAPE: 0.3448275862068966
B Bedrijfskunde, NL, 361, MAE: 20, MAPE: 0.05249343832020997
B Bedrijfskunde, Niet-EER, 11, MAE: 0, MAPE: 0.0
B Bestuurskunde, NL, 97, MAE: 14, MAPE: 0.1686746987951807
B Biology, EER, 94, MAE: 46, MAPE: 0.9583333333333334
B Biology, NL, 92, MAE: 2, MAPE: 0.022222222222222223
B Biology, Niet-EER, 44, MAE: 39, MAPE: 7.8
B Biomedische Wetenschappen, EER, 0, MAE: ???, MAPE: ???
B Biomedische Wetenschappen, NL, 100, MAE: 5, MAPE: 0.05263157894736842
